In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.2.0-beta.4"
#r "nuget: Azure.Identity, 1.15.0"
#r "nuget: System.Linq.Async, 6.0.3"
#r "nuget: Microsoft.Extensions.AI, 9.8.0"
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages Azure.AI.Agents.Persistent, 1.2.0-beta.4 Azure.Identity, 1.15.0 DotNetEnv, 3.1.1 Microsoft.Extensions.AI, 9.8.0 System.Linq.Async, 6.0.3

In [2]:
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.Orchestration\bin\Debug\net9.0\Microsoft.Agents.Orchestration.dll"

In [3]:
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Extensions.AI.Agents.AzureAI\bin\Debug\net9.0\Microsoft.Extensions.AI.Agents.AzureAI.dll"

In [4]:
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Extensions.AI.Agents\bin\Debug\net9.0\Microsoft.Extensions.AI.Agents.dll"

In [5]:
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.Orchestration\bin\Debug\net9.0\Microsoft.Agents.Orchestration.dll"

In [6]:
using System;
using System.Collections.Generic;
using System.IO;
using System.Linq;
using System.Text;
using System.Text.Json;
using System.Threading.Tasks;
using Azure.AI.Agents.Persistent;
using Azure.Identity;
using Microsoft.Extensions.AI;
using Microsoft.Extensions.AI.Agents;
using Microsoft.Extensions.Logging;
using Microsoft.Agents.Orchestration;
using DotNetEnv;

In [7]:
// Load environment variables
Env.Load("../../../.env");

var azure_foundry_endpoint = Environment.GetEnvironmentVariable("FOUNDRY_PROJECT_ENDPOINT") ?? throw new InvalidOperationException("FOUNDRY_PROJECT_ENDPOINT is not set.");
var azure_foundry_model_id = "gpt-4.1";

var bing_conn_id = Environment.GetEnvironmentVariable("BING_CONNECTION_NAME");

In [8]:
var bingGroundingConfig = new BingGroundingSearchConfiguration(bing_conn_id);

BingGroundingToolDefinition bingGroundingTool = new(
    new BingGroundingSearchToolParameters(
        [bingGroundingConfig]
    )
);

In [9]:
// Create the persistent agents client
var persistentAgentsClient = new PersistentAgentsClient(azure_foundry_endpoint, new AzureCliCredential());

In [10]:
const string EvangelistInstructions = @"
You are a technical evangelist, writing a tutorial.
1. The generated content format is markdown
2. Generate a title based on the content
3. Each knowledge point can be accessed through related links to obtain knowledge points and content
4. The content should not include code examples
5. The word count should be no less than 1000 words, and each point needs to be integrated in detail.
";

const string ContentReviewerInstructions = @"
You are a content reviewer for a publishing company. You need to check whether the content meets the following requirements:

1. If code samples are included, they must include both dotNET and Python code. If there are no code samples, skip this requirement.
2. The content should be no less than 1000 words. If it is less than 1000 words, please return no; otherwise, return yes.

If the above requirements are met, return yes; otherwise, return no.

";

const string PublisherInstructions = @"
You are the content publisher and save the content. Saved file's name is marked with current date and time, such as yearmonthdayhourminsec. Note that if it is 1-9, you need to add 0, such as 01.saved the markdown file such as 20240101123045.md. Then Save this content in this file
";

In [11]:
// Create the three specialized agents
var evangelistMetadata = await persistentAgentsClient.Administration.CreateAgentAsync(
    model: azure_foundry_model_id,
    name: "Evangelist",
    instructions: EvangelistInstructions,
    tools: [bingGroundingTool]
);

var contentReviewerMetadata = await persistentAgentsClient.Administration.CreateAgentAsync(
     model: azure_foundry_model_id,
     name: "ContentReviewer", 
     instructions: ContentReviewerInstructions
);

var publisherMetadata = await persistentAgentsClient.Administration.CreateAgentAsync(
    model: azure_foundry_model_id,
    name: "Publisher",
    instructions: PublisherInstructions,
    tools: [new CodeInterpreterToolDefinition()]
);

In [12]:
string evangelist_agentId = evangelistMetadata.Value.Id;
string contentReviewer_agentId = contentReviewerMetadata.Value.Id;
string publisher_agentId = publisherMetadata.Value.Id;

In [13]:
AIAgent evangelistagent = await persistentAgentsClient.GetAIAgentAsync(evangelist_agentId);
AIAgent contentRevieweragent = await persistentAgentsClient.GetAIAgentAsync(contentReviewer_agentId);
AIAgent publisheragent = await persistentAgentsClient.GetAIAgentAsync(publisher_agentId);

In [14]:
IChatClient evangelistChatClient = persistentAgentsClient.AsIChatClient(evangelistagent.Id)
            .AsBuilder()
            .UseFunctionInvocation()
            .Build();

IChatClient contentReviewerChatClient = persistentAgentsClient.AsIChatClient(contentRevieweragent.Id)
            .AsBuilder()
            .UseFunctionInvocation()
            .Build();

IChatClient publisherChatClient = persistentAgentsClient.AsIChatClient(publisheragent.Id)
            .AsBuilder()
            .UseFunctionInvocation()
            .Build();

In [15]:

ChatClientAgent evangelistAgent = new ChatClientAgent(evangelistChatClient);
ChatClientAgent contentReviewerAgent = new ChatClientAgent(contentReviewerChatClient);
ChatClientAgent publisherAgent = new ChatClientAgent(publisherChatClient);

In [16]:
public  class OrchestrationMonitor
{
        /// <summary>
        /// Gets the list of streamed response updates received so far.
        /// </summary>
        public List<AgentRunResponseUpdate> StreamedResponses { get; } = [];

        /// <summary>
        /// Gets the list of chat messages representing the conversation history.
        /// </summary>
        public List<ChatMessage> History { get; } = [];

        /// <summary>
        /// Callback to handle a batch of chat messages, adding them to history and writing them to output.
        /// </summary>
        /// <param name="response">The collection of <see cref="ChatMessage"/> objects to process.</param>
        /// <returns>A <see cref="ValueTask"/> representing the asynchronous operation.</returns>
        public ValueTask ResponseCallback(IEnumerable<ChatMessage> response)
        {
            this.WriteStreamedResponse(this.StreamedResponses);
            this.StreamedResponses.Clear();

            this.History.AddRange(response);
            this.WriteResponse(response);
            return default;
        }

        /// <summary>
        /// Callback to handle a streamed agent run response update, adding it to the list and writing output if final.
        /// </summary>
        /// <param name="streamedResponse">The <see cref="AgentRunResponseUpdate"/> to process.</param>
        /// <returns>A <see cref="ValueTask"/> representing the asynchronous operation.</returns>
        public ValueTask StreamingResultCallback(AgentRunResponseUpdate streamedResponse)
        {
            this.StreamedResponses.Add(streamedResponse);
            return default;
        }

        
        public void WriteResponse(IEnumerable<ChatMessage> response)
        {
            foreach (ChatMessage message in response)
            {
                if (!string.IsNullOrEmpty(message.Text))
                {
                    System.Console.WriteLine($"\n# RESPONSE {message.Role}{(message.AuthorName is not null ? $" - {message.AuthorName}" : string.Empty)}: {message}");
                }
            }
        }

        /// <summary>
        /// Writes the streamed agent run response updates to the console or test output, including role and author information.
        /// </summary>
        /// <param name="streamedResponses">An enumerable of <see cref="AgentRunResponseUpdate"/> objects representing streamed responses.</param>
        public void WriteStreamedResponse(IEnumerable<AgentRunResponseUpdate> streamedResponses)
        {
            string? authorName = null;
            ChatRole? authorRole = null;
            StringBuilder builder = new();
            foreach (AgentRunResponseUpdate response in streamedResponses)
            {
                authorName ??= response.AuthorName;
                authorRole ??= response.Role;

                if (!string.IsNullOrEmpty(response.Text))
                {
                    builder.Append($"({JsonSerializer.Serialize(response.Text)})");
                }
            }

            if (builder.Length > 0)
            {
                System.Console.WriteLine($"\n# STREAMED {authorRole ?? ChatRole.Assistant}{(authorName is not null ? $" - {authorName}" : string.Empty)}: {builder}\n");
            }
        }
}

In [17]:
 OrchestrationMonitor monitor = new();

In [18]:
ILoggerFactory LoggerFactory { get; }

In [19]:
GroupChatOrchestration orchestration =
            new(
                new RoundRobinGroupChatManager()
                {
                    MaximumInvocationCount = 3
                },
                evangelistAgent,
                contentReviewerAgent,
                publisherAgent
            )
            {
                //LoggerFactory =  LoggerFactory.Create(builder => builder.AddConsole().SetMinimumLevel(LogLevel.Warning)),
                ResponseCallback = monitor.ResponseCallback,
                StreamingResponseCallback = monitor.StreamingResultCallback,
            };


In [20]:
string[] files= [
    "C:\\Users\\kinfeylo\\Documents\\Agent\\AgentframeworkInAction\\01.AgentFoundation\\agent-framework-workflow-architecture.md",
    "C:\\Users\\kinfeylo\\Documents\\Agent\\AgentframeworkInAction\\01.AgentFoundation\\README.md"
];

In [21]:
// Method to get file contents from local or remote sources
async Task<string> GetFileContents(string filePath)
{
    try
    {
        if (File.Exists(filePath))
        {
            Console.WriteLine($"📁 Reading local file: {filePath}");
            return await File.ReadAllTextAsync(filePath);
        }
        else
        {
            throw new FileNotFoundException($"File not found: {filePath}");
        }
    }
    catch (Exception ex)
    {
        Console.WriteLine($"❌ Error reading file {filePath}: {ex.Message}");
        return $"Error reading file: {ex.Message}";
    }
}


In [22]:
// Method to get multiple file contents at once
async Task<Dictionary<string, string>> GetMultipleFileContents(params string[] filePaths)
{
    var results = new Dictionary<string, string>();
    
    Console.WriteLine($"📚 Processing {filePaths.Length} files...");
    
    foreach (string filePath in filePaths)
    {
        try
        {
            string content = await GetFileContents(filePath);
            results[filePath] = content;
            Console.WriteLine($"✅ Successfully loaded: {Path.GetFileName(filePath)}");
        }
        catch (Exception ex)
        {
            results[filePath] = $"Error: {ex.Message}";
            Console.WriteLine($"❌ Failed to load: {filePath}");
        }
    }
    
    return results;
}

In [23]:
string[] files= [
    "C:\\Users\\kinfeylo\\Documents\\Agent\\AgentframeworkInAction\\01.AgentFoundation\\agent-framework-workflow-architecture.md",
    "C:\\Users\\kinfeylo\\Documents\\Agent\\AgentframeworkInAction\\01.AgentFoundation\\README.md"
];

In [24]:
var content = await GetMultipleFileContents(files);

📚 Processing 2 files...
📁 Reading local file: C:\Users\kinfeylo\Documents\Agent\AgentframeworkInAction\01.AgentFoundation\agent-framework-workflow-architecture.md
✅ Successfully loaded: agent-framework-workflow-architecture.md
📁 Reading local file: C:\Users\kinfeylo\Documents\Agent\AgentframeworkInAction\01.AgentFoundation\README.md
✅ Successfully loaded: README.md


In [25]:
ChatMessage userMessage = new ChatMessage(ChatRole.User, [
	new TextContent("Complete the tutorial according to the provided outline content and related links, then review the content，if all is well, please Save it as markdown file. If not,please recreate it. When all steps are complete, please let me know."),new TextContent(content["C:\\Users\\kinfeylo\\Documents\\Agent\\AgentframeworkInAction\\01.AgentFoundation\\README.md"])
]);

In [26]:
AgentRunResponse result = await orchestration.RunAsync(userMessage);


# STREAMED assistant - asst_xbSNDFjRU6T4Zv8hGNLYSm73: ("#")(" Introducing")(" AI")(" Agents")(":")(" Concepts")(",")(" Services")(",")(" and")(" Framework")("s")("\n\n")("Artificial")(" Intelligence")(" (")("AI")(")")(" continues")(" to")(" rapidly")(" evolve")(",")(" and")(" one")(" of")(" the")(" most")(" transformative")(" aspects")(" is")(" the")(" emergence")(" of")(" AI")(" agents")(".")(" These")(" entities")(" can")(" analyze")(" information")(",")(" respond")(" intelligently")(",")(" and")(" even")(" interact")(" autonom")("ously")(" within")(" digital")(" and")(" physical")(" environments")(".")(" In")(" this")(" tutorial")(",")(" you")(" will")(" gain")(" a")(" comprehensive")(" understanding")(" of")(" what")(" AI")(" agents")(" are")(",")(" familiarize")(" yourself")(" with")(" the")(" Azure")(" AI")(" Found")("ry")(" Agent")(" Service")(",")(" and")(" explore")(" Microsoft\u0027s")(" Agent")(" Framework")(".")(" Each")(" section")(" will")(" be")(" linked")(" to")(" in")

In [27]:
result

The tutorial content has been successfully saved as a markdown file:\n\n**File name:** `20250916141443.md`\n\n**Location:** `/mnt/data/20250916141443.md`\n\nAll steps have been completed! If you need to download the file, you can [click here to access it directly](sandbox:/mnt/data/20250916141443.md... Messages index value 0 The tutorial content has been successfully saved as a markdown file:\n\n**File name:** `20250916141443.md`\n\n**Location:** `/mnt/data/20250916141443.md`\n\nAll steps have been completed! If you need to download the file, you can [click here to access it directly](sandbox:/mnt/data/20250916141443.md... AuthorName acd3a140-c7f1-400a-accb-70cae52beab4 CreatedAt <null> Role assistant Value assistant Text The tutorial content has been successfully saved as a markdown file:

**File name:** `20250916141443.md`

**Location:** `/mnt/data/20250916141443.md`

All steps have been completed! If you need to download the file, you can [click here to access it directly](sandbox:/mnt/data/20250916141443.md). If you require additional edits or further assistance, just let me know! Contents index value 0 The tutorial content has been successfully saved as a markdown file:\n\n**File name:** `20250916141443.md`\n\n**Location:** `/mnt/data/20250916141443.md`\n\nAll steps have been completed! If you need to download the file, you can [click here to access it directly](sandbox:/mnt/data/20250916141443.md... Text The tutorial content has been successfully saved as a markdown file:

**File name:** `20250916141443.md`

**Location:** `/mnt/data/20250916141443.md`

All steps have been completed! If you need to download the file, you can [click here to access it directly](sandbox:/mnt/data/20250916141443.md). If you require additional edits or further assistance, just let me know! Annotations <null> RawRepresentation <null> AdditionalProperties <null> MessageId <null> RawRepresentation <null> AdditionalProperties <null> Text The tutorial content has been successfully saved as a markdown file:

**File name:** `20250916141443.md`

**Location:** `/mnt/data/20250916141443.md`

All steps have been completed! If you need to download the file, you can [click here to access it directly](sandbox:/mnt/data/20250916141443.md). If you require additional edits or further assistance, just let me know! UserInputRequests (empty) AgentId <null> ResponseId <null> CreatedAt <null> Usage <null> RawRepresentation <null> AdditionalProperties <null>

In [28]:
public  string? AsJson(object? obj)
{
    JsonSerializerOptions s_jsonOptionsCache = new() { WriteIndented = true };
        if (obj is null) { return null; }
        return JsonSerializer.Serialize(obj, s_jsonOptionsCache);
}


In [29]:

    public void WriteMessageOutput(ChatMessage message)
    {
        string authorExpression = message.Role == ChatRole.User ? string.Empty : FormatAuthor();
        string contentExpression = message.Text.Trim();
        bool isCode = false; //message.AdditionalProperties?.ContainsKey(OpenAIAssistantAgent.CodeInterpreterMetadataKey) ?? false;
        string codeMarker = isCode ? "\n  [CODE]\n" : " ";
        Console.WriteLine($"\n# {message.Role}{authorExpression}:{codeMarker}{contentExpression}");

        // Provide visibility for inner content (that isn't TextContent).
        foreach (AIContent item in message.Contents)
        {
            if(item is TextContent file)
            { 
                if(file.Annotations is not null && file.Annotations.Count > 0)
                {
                    var annotation = file.Annotations[0];
                    if(annotation.GetType().Name == "MessageTextFilePathAnnotation")
                    {
                        Console.WriteLine("qqxx------------------------------------------------");
                    }   
                }
            }
            /*
            if(file.Annotations[0] is HostedFileContent pathItem )
            {
                Console.WriteLine(pathItem.FileId);

                BinaryData filecontent =  persistentAgentsClient.Files.GetFileContent(pathItem.FileId);
                string tempFilePath1 = "c:\\Users\\kinfeylo\\Documents\\Agent\\AgentframeworkInAction\\07.MultiAgents\\code_samples\\dotNET\\outputs\\" + GetFileName(pathItem.Text);
                File.WriteAllBytes(tempFilePath1, filecontent.ToArray());
                persistentAgentsClient.Files.DeleteFile(pathItem.FileId);

                ProcessStartInfo psi1 = new()
                {
                    FileName = tempFilePath1,
                    UseShellExecute = true
                };
                Process.Start(psi1);
            }
            */
            /*
            if (item is DataContent image && image.HasTopLevelMediaType("image"))
            {
                Console.WriteLine($"  [{item.GetType().Name}] {image.Uri?.ToString() ?? image.Uri ?? $"{image.Data.Length} bytes"}");
            }
            else if( item is MessageTextContent file)
            {
                Console.WriteLine($"  [{item.GetType().Name}] {file.Uri?.ToString() ?? file.Uri ?? $"{file.Data.Length} bytes"}");
            }
            else if (item is FunctionCallContent functionCall)
            {
                Console.WriteLine($"  [{item.GetType().Name}] {functionCall.CallId}");
            }
            else if (item is FunctionResultContent functionResult)
            {
                Console.WriteLine($"  [{item.GetType().Name}] {functionResult.CallId} - {AsJson(functionResult.Result) ?? "*"}");
            }
            */
        }

        string FormatAuthor() => message.AuthorName is not null ? $" - {message.AuthorName ?? " * "}" : string.Empty;
    }  
    

In [30]:
void DisplayHistory(IEnumerable<ChatMessage> history)
{
        Console.WriteLine("\n\nORCHESTRATION HISTORY");
        foreach (ChatMessage message in history)
        {
            WriteMessageOutput(message);
        }
}

In [31]:
DisplayHistory(monitor.History);



ORCHESTRATION HISTORY

# assistant - asst_xbSNDFjRU6T4Zv8hGNLYSm73: # Introducing AI Agents: Concepts, Services, and Frameworks

Artificial Intelligence (AI) continues to rapidly evolve, and one of the most transformative aspects is the emergence of AI agents. These entities can analyze information, respond intelligently, and even interact autonomously within digital and physical environments. In this tutorial, you will gain a comprehensive understanding of what AI agents are, familiarize yourself with the Azure AI Foundry Agent Service, and explore Microsoft's Agent Framework. Each section will be linked to in-depth, authoritative resources for further exploration.

---

## What’s an AI Agent?

**AI agents** are autonomous systems that perceive their environment, process inputs, and take actions to achieve specific objectives. They can range from simple software bots to complex cognitive agents interacting in dynamic real-world environments.

### **Key Concepts of AI Agents**
1. **A